# Coursera Applied Data Science Capstone
This notebook contains my work for the applied data sience capstone course from IBM/Coursera.

# Introduction

One of the most difficult things about moving to a new city on the other side of the world is deciding where to live. While you can look-up online the characteristics of the most famous suburbs many others that could be hidden gems are overlooked. So, what if you could use the place you are currently living in as a reference to look for recommendations of where to live on the other side of the world. This recommendation could include suburb characteristics such as the types of businesses and facilities around it and some statistical information on the rent prices. Thus, narrowing down your search for the perfect place to live on the other side of the world!
This service is aimed at the people of our globalised world, who are looking to move to a new city within their country or internationally. For this project I'll use myself as an example, back in in the day I move from Guadalajara, MX to Melbourne, AU. So I'll be using the place I used to live in Mexico to rank suburbs in Melbourne that could be suited to me.

# Data

This is the data I plan to use for my solution:
## Venue data for a specific suburb in Guadalajara, MX.  
The location (lat, lon) for the specific suburb will need to be determined for the Foursquare queries, this can be easily obtained through a Google search.  
This location data will be used to obtain venue information using the Foursquare API. The focus will be in obtaining the top 5 venue categories by frequency in that area.  
## Rent price data for all suburbs in Guadalajara, MX.  
This is probably the most difficult to obtain as Mexico is not great with data gathering. The objective with this data is to obtain the percentile the specific suburb is in w.r.t. rent prices. This is in order to have a better price comparison between the two cities as they may be quite different in terms of cost of living. This is with the assumption that the person is (at least) looking at maintaining their current living conditions.  
Thus, the median rental price for housing properties in all the suburbs of Guadalajara, MX are needed.
## Venue data for all Melbourne, AU suburbs.
Location data for all the Melbourne suburbs is needed for the Foursquare venue queries. This can be easily obtained from the Australian government's data access website: https://data.gov.au/dataset/ds-dga-af33dd8c-0534-4e18-9245-fc64440f742e/details  
Similarly to the venue information discussed before, the top 5 venue categories will be required for all Melbourne suburbs. The venue data will be used in the comparison between the two cities (by suburb).
## Rent price data for all suburbs in Melbourne, AU.  
Median rental price data for all the Melbourne suburbs can be obtained from the Victorian government's website: https://www.dhhs.vic.gov.au/publications/rental-report.  
This data will be used to determine each suburb's rent percentile, which will then be used as an additional feature for suburb comparison.

#### Imports
All the stuff needed for the project.

In [1]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests

# Source suburb data

## Geographical location

Let's use geolocator to get the geographical coordinates of the source suburb of interest.

In [2]:
source_suburb = 'providencia'
source_city = 'guadalajara, mexico'
address = source_suburb + ', ' + source_city

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
source_loc = [latitude, longitude]
print('The geograpical coordinates of {} are {}, {}.'.format(source_suburb.upper(), latitude, longitude))

The geograpical coordinates of PROVIDENCIA are 20.6973972, -103.3786781.


Let's show it in the map

In [3]:
# create map of New York using latitude and longitude values
source_map = folium.Map(location=source_loc, zoom_start=16)
source_map

That looks about right!

## Rent price data

After a good time searching, I found a database containing rental information for the whole city of Guadalajara at the following link:
https://iieg.gob.mx/ns/?page_id=11967  
I've downloaded the spreadsheet and cleaned it up so I can easily import it, let's check it out.

In [4]:
source_rent_df = pd.read_csv('Data/Clean/Guadalajara_Rent_Data.csv')
source_rent_df.head()

,Type,Rent,City,Suburb
0,Departamento,7000.0,Guadalajara,LAS TORRES
1,Casa,15000.0,Tlajomulco,BOSQUE REAL DE SANTA ANITA
2,Casa,15000.0,Tlajomulco,BOSQUE REAL DE SANTA ANITA
3,Departamento,16500.0,Tlajomulco,LA RIOJA
4,Departamento,17500.0,Guadalajara,JARDINES DEL BOSQUE


Alright, so we are interested in suburb and rent prices. Now, this dataset contains multiple entries per suburb as it reflects the current market. Let's first start by only having suburb and rent in the dataframe and changing the suburb names to uppercase.

In [5]:
source_rent_df = source_rent_df[['Suburb', 'Rent']] # Keep only Suburb and Rent
source_rent_df['Suburb'] = source_rent_df['Suburb'].str.upper() # Lower case
source_rent_df.head()

,Suburb,Rent
0,LAS TORRES,7000.0
1,BOSQUE REAL DE SANTA ANITA,15000.0
2,BOSQUE REAL DE SANTA ANITA,15000.0
3,LA RIOJA,16500.0
4,JARDINES DEL BOSQUE,17500.0


Now let's group by suburn and calculate the mean rent price.

In [6]:
source_rent_df = source_rent_df.groupby(['Suburb']).mean()
source_rent_df.reset_index(inplace=True)
source_rent_df.head()

,Suburb,Rent
0,AGRARIA,13000.000000
1,ALBATERRA,7250.000000
2,ALTAMIRA,23250.000000
3,ALTEA,7000.000000
4,AMERICANA,20026.136364


Looking good, let's check some basic stats as a sanity check.

In [7]:
source_rent_stats = source_rent_df.describe()
source_rent_stats

,Rent
count,317.000000
mean,17272.443272
std,9935.217925
min,1800.000000
25%,9500.000000
50%,15500.000000
75%,23000.000000
max,65000.000000


Since the source and targets may be in diferent countries, the rent prices will be in different currency. Also, the cost of living may be different. Thus, we need a relative measure of the rent price that can be used for comparison. I chose to use the percentile. Let's calculate what percentile does the source suburb belong to.

In [8]:
def calc_percentile(max_value, min_value, value):
    return 100*((value - min_value)/(max_value-min_value))

In [9]:
# Create a percentile column for the source dataframe
source_rent_df['Percentile'] = calc_percentile(source_rent_df.max()['Rent'], source_rent_df.min()['Rent'], source_rent_df['Rent'])
source_rent_df.head()

,Suburb,Rent,Percentile
0,AGRARIA,13000.000000,17.721519
1,ALBATERRA,7250.000000,8.623418
2,ALTAMIRA,23250.000000,33.939873
3,ALTEA,7000.000000,8.227848
4,AMERICANA,20026.136364,28.838823


In [10]:
# Get the source median rent price
source_rent_df[source_rent_df['Suburb']==source_suburb.upper()]

,Suburb,Rent,Percentile
313,PROVIDENCIA,26000.0,38.291139


## Venue data

Now we just need to collect venue information for the source suburb, we'll use the Foursquare API for this.

In [11]:
CLIENT_ID = '543RCQIDW44OE0ZDAPA5H5I3WDPAQTJZBSA1IIRNMYOV2B4W' # your Foursquare ID
CLIENT_SECRET = 'HTU223QZD4GOZ3W2H424YP2CZ1LGKOF1KSYNKICNEOYAV1V2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 543RCQIDW44OE0ZDAPA5H5I3WDPAQTJZBSA1IIRNMYOV2B4W
CLIENT_SECRET:HTU223QZD4GOZ3W2H424YP2CZ1LGKOF1KSYNKICNEOYAV1V2


#### Define function to explore neighborhoods

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now run the above function on each neighborhood and create a new dataframe called *source_venues*.

In [13]:
source_explore_data = pd.DataFrame({'Suburb':[source_suburb.upper()], 'Latitude':[source_loc[0]], 'Longitude':[source_loc[1]]})
source_venues = getNearbyVenues(names=source_explore_data['Suburb'],
                                 latitudes=source_explore_data['Latitude'],
                                 longitudes=source_explore_data['Longitude'], radius = 500)

PROVIDENCIA


#### Let's check the size of the resulting dataframe

In [14]:
print(source_venues.shape)
source_venues.head()

(61, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,PROVIDENCIA,20.697397,-103.378678,Sensecycle,20.698501,-103.377518,Other Event
1,PROVIDENCIA,20.697397,-103.378678,The Blooming Tea,20.697546,-103.380022,Tea Room
2,PROVIDENCIA,20.697397,-103.378678,Parque Dr. Atl,20.695867,-103.378644,Garden
3,PROVIDENCIA,20.697397,-103.378678,The Barre Studio,20.698223,-103.377308,Gym / Fitness Center
4,PROVIDENCIA,20.697397,-103.378678,Anytime Fitness Ottawa,20.698181,-103.377353,Gym / Fitness Center


#### Let's find out how many unique categories can be curated from all the returned venues

In [15]:
print('There are {} uniques categories.'.format(len(source_venues['Venue Category'].unique())))

There are 45 uniques categories.


### Suburb analysis

In [16]:
# Start with one hot encoding
source_onehot = pd.get_dummies(source_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
source_onehot['Neighborhood'] = source_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [source_onehot.columns[-1]] + list(source_onehot.columns[:-1])
source_onehot = source_onehot[fixed_columns]

source_onehot.head()

,Neighborhood,Argentinian Restaurant,Arts & Crafts Store,Asian Restaurant,Boat or Ferry,Café,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,...,Shopping Mall,Snack Place,Spa,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Toy / Game Store,Wings Joint,Yoga Studio
0,PROVIDENCIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,PROVIDENCIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,PROVIDENCIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,PROVIDENCIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,PROVIDENCIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [17]:
source_onehot.shape

(61, 46)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [18]:
source_grouped = source_onehot.groupby('Neighborhood').mean().reset_index()
source_grouped

,Neighborhood,Argentinian Restaurant,Arts & Crafts Store,Asian Restaurant,Boat or Ferry,Café,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,...,Shopping Mall,Snack Place,Spa,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Toy / Game Store,Wings Joint,Yoga Studio
0,PROVIDENCIA,0.016393,0.016393,0.016393,0.016393,0.032787,0.016393,0.065574,0.016393,0.016393,...,0.016393,0.016393,0.032787,0.016393,0.016393,0.016393,0.016393,0.016393,0.016393,0.032787


#### Let's confirm the new size

In [19]:
source_grouped.shape

(1, 46)

#### Let's print the top 5 most common venues

In [20]:
num_top_venues = 5

for hood in source_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = source_grouped[source_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----PROVIDENCIA----
                  venue  freq
0            Restaurant  0.07
1           Coffee Shop  0.07
2             Nightclub  0.05
3  Gym / Fitness Center  0.05
4        Ice Cream Shop  0.03




#### Let's put the top venues into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
source_venues_sorted = pd.DataFrame(columns=columns)
source_venues_sorted['Neighborhood'] = source_grouped['Neighborhood']

for ind in np.arange(source_grouped.shape[0]):
    source_venues_sorted.iloc[ind, 1:] = return_most_common_venues(source_grouped.iloc[ind, :], num_top_venues)

source_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PROVIDENCIA,Restaurant,Coffee Shop,Gym / Fitness Center,Nightclub,Yoga Studio,Café,Deli / Bodega,Mexican Restaurant,Ice Cream Shop,Spa


#### Combine with rent cost percentile

In [24]:
source_venues_sorted['Rent Percentile'] = source_rent_df[source_rent_df['Suburb']==source_suburb.upper()]['Percentile']
source_venues_sorted.at[0, 'Rent Percentile'] = source_rent_df[source_rent_df['Suburb']==source_suburb.upper()]['Percentile']
source_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Rent Percentile
0,PROVIDENCIA,Restaurant,Coffee Shop,Gym / Fitness Center,Nightclub,Yoga Studio,Café,Deli / Bodega,Mexican Restaurant,Ice Cream Shop,Spa,38.291139


Now we have all the data from the source suburb to be able to compare it to suburbs in the target city.

# Target City Data